# Version
* `v13`: Fold4
* `v12`: Fold3
* `v10`: Fold2
* `v09`: Fold1
* `v03`: Fold0

In [ ]:
!pip install --upgrade seaborn

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns

In [ ]:
dim = '512' #512, 256, 'original'
fold = 4

In [ ]:
train_df = pd.read_csv(f'../input/vinbigdata-{dim}-image-dataset/vinbigdata/train.csv')
# train_df = pd.read_csv(f'../input/vinbigdata-512-image-dataset/vinbigdata/train.csv')

train_df.head()

In [ ]:
train_df['image_path'] = f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/train/'+train_df.image_id+('.png' if dim!='original' else '.jpg')
train_df.head()

# Only 14 Class

In [ ]:
without_mass=train_df[train_df.class_id!=8]

without_mass_files   = []
without_mass_files += list(without_mass.image_path.unique())

In [ ]:
train_df = train_df[train_df.class_id==8].reset_index(drop = True)

# Split

In [ ]:
!pip install ensemble_boxes

In [ ]:
gkf  = GroupKFold(n_splits = 5)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups = train_df.image_id.tolist())):
    train_df.loc[val_idx, 'fold'] = fold
train_df.head()

In [ ]:
train_files = []
val_files   = []
val_files += list(train_df[train_df.fold==fold].image_path.unique())
train_files += list(train_df[train_df.fold!=fold].image_path.unique())
len(train_files), len(val_files)

In [ ]:
# os.makedirs('/kaggle/working/vinbigdata/labels/train', exist_ok = True)
# os.makedirs('/kaggle/working/vinbigdata/labels/val', exist_ok = True)
# os.makedirs('/kaggle/working/vinbigdata/images/train', exist_ok = True)
# os.makedirs('/kaggle/working/vinbigdata/images/val', exist_ok = True)
# label_dir = '/kaggle/input/vinbigdata-yolo-labels-dataset/labels'

In [ ]:
import os.path as osp
from path import Path
from collections import Counter
import cv2
from ensemble_boxes import *

In [ ]:
imagepaths = train_df['image_path'].unique()
train_annotations=train_df

In [ ]:
train_annotations['area']=((train_annotations['x_max']-train_annotations['x_min'])/train_annotations['width'])*((train_annotations['y_max']-train_annotations['y_min'])/train_annotations['height'])

In [ ]:
# img_array  = cv2.imread('/kaggle/input/vinbigdata-512-image-dataset/vinbigdata/train/d3637a1935a905b3c326af31389cb846.png')

In [ ]:
# def Create_nms_box_txt(desktop_path,name):
#     iou_thr = 0.5
#     skip_box_thr = 0.0001
#     viz_images = []
# #     image_basename = Path(path).stem
#     image_basename = name
# #     print(f"(\'{image_basename}\', \'{path}\')")
#     img_annotations = train_annotations[train_annotations.image_id==image_basename][train_annotations.class_id==8]

#     boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
#     labels_viz = img_annotations['class_id'].to_numpy().tolist()

#     print("Bboxes before nms:\n", boxes_viz)
#     print("Labels before nms:\n", labels_viz)

#     boxes_list = []
#     scores_list = []
#     labels_list = []
#     weights = []

#     boxes_single = []
#     labels_single = []

#     cls_ids = img_annotations['class_id'].unique().tolist()
#     count_dict = Counter(img_annotations['class_id'].tolist())
#     print(count_dict)

#     for cid in cls_ids:       
#         ## Performing Fusing operation only for multiple bboxes with the same label
#         if count_dict[cid]==1:
#             labels_single.append(cid)
#             boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

#         else:
#             cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
#             labels_list.append(cls_list)
#             bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

#             ## Normalizing Bbox by Image Width and Height
#             bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


#             bbox = np.clip(bbox, 0, 1)
#             boxes_list.append(bbox.tolist())

#             scores_list.append(np.ones(len(cls_list)).tolist())

#             weights.append(1)

            
#     # Perform NMS
#     if len(boxes_list)==0:
#         boxes=boxes_single
#         box_labels=labels_single
#         print("Bboxes after nms:\n", boxes)
#         print("Labels after nms:\n", box_labels)
        
#         count_dict = Counter(box_labels)
#         print(count_dict)

#         text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])
        
#     else:
#         boxes, scores, box_labels = nms(boxes_list, scores_list, labels_list, weights=weights,
#                                     iou_thr=iou_thr)


#         #img_array.shape[1]是宽度
#         boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
#         boxes = boxes.round(1).tolist()
#         box_labels = box_labels.astype(int).tolist()

#         boxes.extend(boxes_single)
#         box_labels.extend(labels_single)

#         print("Bboxes after nms:\n", boxes)
#         print("Labels after nms:\n", box_labels)

#         count_dict = Counter(box_labels)
#         print(count_dict)

#         text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
# iou_thr = 0.5
# skip_box_thr = 0.0001
# viz_images = []
# for i, path in tqdm(enumerate(imagepaths[5:6])):
#     image_basename = Path(path).stem
#     print(f"(\'{image_basename}\', \'{path}\')")
#     img_annotations = train_annotations[train_annotations.image_id==image_basename]

#     boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
#     labels_viz = img_annotations['class_id'].to_numpy().tolist()

#     print("Bboxes before nms:\n", boxes_viz)
#     print("Labels before nms:\n", labels_viz)

#     boxes_list = []
#     scores_list = []
#     labels_list = []
#     weights = []

#     boxes_single = []
#     labels_single = []

#     cls_ids = img_annotations['class_id'].unique().tolist()
#     count_dict = Counter(img_annotations['class_id'].tolist())
#     print(count_dict)

#     for cid in cls_ids:       
#         ## Performing Fusing operation only for multiple bboxes with the same label
#         if count_dict[cid]==1:
#             labels_single.append(cid)
#             boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

#         else:
#             cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
#             labels_list.append(cls_list)
#             bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

#             ## Normalizing Bbox by Image Width and Height
#             bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


#             bbox = np.clip(bbox, 0, 1)
#             boxes_list.append(bbox.tolist())

#             scores_list.append(np.ones(len(cls_list)).tolist())

#             weights.append(1)


#     # Perform NMS
#     boxes, scores, box_labels = nms(boxes_list, scores_list, labels_list, weights=weights,
#                                     iou_thr=iou_thr)
    
#     print("Bboxes without multipy:\n", boxes)

#     #img_array.shape[1]是宽度
#     boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
#     boxes = boxes.round(1).tolist()
#     box_labels = box_labels.astype(int).tolist()

#     boxes.extend(boxes_single)
#     box_labels.extend(labels_single)

#     print("Bboxes after nms:\n", boxes)
#     print("Labels after nms:\n", box_labels)

#     count_dict = Counter(box_labels)
#     print(count_dict)

#     text_create('./',image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
train_annotations=train_annotations[train_annotations.area<0.026]

In [ ]:
train_annotations

In [ ]:
def Create_softnms_box_txt(desktop_path,name):
    iou_thr = 0.5
    skip_box_thr = 0.0001
    viz_images = []
    sigma = 0.1
#     image_basename = Path(path).stem
    image_basename = name
#     print(f"(\'{image_basename}\', \'{path}\')")
    img_annotations = train_annotations[train_annotations.image_id==image_basename][train_annotations.class_id==8]

    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz_raw = img_annotations['class_id'].to_numpy().tolist()
    labels_viz = [0 if wd == 8 else wd for wd in labels_viz_raw]
    
    print('boxes_viz',boxes_viz)
    
    if(len(boxes_viz)==0):
        text_create(desktop_path,image_basename,[],[],0,0)
        return None

    print('没有return')
        
    print("Bboxes before nms:\n", boxes_viz)
    print("Labels before nms:\n", labels_viz)

    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []

    boxes_single = []
    labels_single = []

    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())
    print(count_dict)

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())

            scores_list.append(np.ones(len(cls_list)).tolist())

            weights.append(1)

            

    # Perform NMS
    if len(boxes_list)==0:
        boxes=boxes_single
        box_labels=labels_single
        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)
        
        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])
        
    else:
        boxes, scores, box_labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights,
                                    iou_thr=iou_thr)


        #img_array.shape[1]是宽度
        boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
        boxes = boxes.round(1).tolist()
        box_labels = box_labels.astype(int).tolist()

        boxes.extend(boxes_single)
        box_labels.extend(labels_single)

        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)

        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
def Create_non_maximum_weighted_box_txt(desktop_path,name):
    iou_thr = 0.5
    skip_box_thr = 0.0001
    viz_images = []
#     image_basename = Path(path).stem
    image_basename = name
#     print(f"(\'{image_basename}\', \'{path}\')")
    img_annotations = train_annotations[train_annotations.image_id==image_basename][train_annotations.class_id==8]

    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz = img_annotations['class_id'].to_numpy().tolist()

    print("Bboxes before nms:\n", boxes_viz)
    print("Labels before nms:\n", labels_viz)

    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []

    boxes_single = []
    labels_single = []

    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())
    print(count_dict)

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())

            scores_list.append(np.ones(len(cls_list)).tolist())

            weights.append(1)

            
    # Perform NMS
    if len(boxes_list)==0:
        boxes=boxes_single
        box_labels=labels_single
        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)
        
        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])
        
    else:
        boxes, scores, box_labels = non_maximum_weighted(boxes_list, scores_list, labels_list, weights=weights,
                                    iou_thr=iou_thr)


        #img_array.shape[1]是宽度
        boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
        boxes = boxes.round(1).tolist()
        box_labels = box_labels.astype(int).tolist()

        boxes.extend(boxes_single)
        box_labels.extend(labels_single)

        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)

        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
# os.makedirs('/kaggle/working/vinbigdata/labels/train', exist_ok = True)
# os.makedirs('/kaggle/working/vinbigdata/labels/val', exist_ok = True)
# os.makedirs('/kaggle/working/vinbigdata/images/train', exist_ok = True)
# os.makedirs('/kaggle/working/vinbigdata/images/val', exist_ok = True)
# label_dir = '/kaggle/input/vinbigdata-yolo-labels-dataset/labels'

In [ ]:
# # shutil.copy(file, '/kaggle/working/vinbigdata/images/train')

# img_annotations = train_df[train_df.image_id=='47ed17dcb2cbeec15182ed335a8b5a9e'][train_df.class_id==8]

In [ ]:
# img_annotations

In [ ]:
def Create_weighted_boxes_fusion_box_txt(desktop_path,name):
    iou_thr = 0.5
    skip_box_thr = 0.0001
    viz_images = []
#     image_basename = Path(path).stem
    image_basename = name
#     print(f"(\'{image_basename}\', \'{path}\')")
    img_annotations = train_annotations[train_annotations.image_id==image_basename][train_df.class_id==8]

    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz_raw = img_annotations['class_id'].to_numpy().tolist()
    labels_viz = [0 if wd == 8 else wd for wd in labels_viz_raw]

    print("Bboxes before nms:\n", boxes_viz)
    print("Labels before nms:\n", labels_viz)

    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []

    boxes_single = []
    labels_single = []

    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())
    print(count_dict)

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())

            scores_list.append(np.ones(len(cls_list)).tolist())

            weights.append(1)

            
    # Perform NMS
    if len(boxes_list)==0:
        boxes=boxes_single
        box_labels=labels_single
        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)
        
        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])
        
    else:
        boxes, scores, box_labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights,
                                    iou_thr=iou_thr)


        #img_array.shape[1]是宽度
        boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
        boxes = boxes.round(1).tolist()
        box_labels = box_labels.astype(int).tolist()

        boxes.extend(boxes_single)
        box_labels.extend(labels_single)

        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)

        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
def text_create(desktop_path,name, boxes,box_labels,w,h):
    
        
    full_path = os.path.join(desktop_path, name+'.txt')  # 也可以创建一个.doc的word文档
    print('full_path',full_path)
    file = open(full_path, 'w')
    
    if(len(boxes)==0):
        file.close()
    else:
        dw = 1. / (w)
        dh = 1. / (h)

        for i in range(len(boxes)):


            x = (boxes[i][0] + boxes[i][2]) / 2.0
            y = (boxes[i][1] + boxes[i][3]) / 2.0
            w = boxes[i][2] - boxes[i][0]
            h = boxes[i][3] - boxes[i][1]


            x = x * dw
            w = w * dw
            y = y * dh
            h = h * dh


    #         file.write(str(box_labels[i])+ ' '+ str(x)+ ' '+ str(y)+ ' '+ str(w)+ ' '+ str(h)+ ' '+ '\n') 
            file.write(str(0)+ ' '+ str(x)+ ' '+ str(y)+ ' '+ str(w)+ ' '+ str(h)+ ' '+ '\n')
            print('录入'+str(0)+ ' '+ str(x)+ ' '+ str(y)+ ' '+ str(w)+ ' '+ str(h)+ ' ')
        file.close()

# Copying Files

In [ ]:
train_files

In [ ]:
os.makedirs('/kaggle/working/vinbigdata/labels/train', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/labels/val', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/images/train', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/images/val', exist_ok = True)
label_dir = '/kaggle/input/vinbigdata-yolo-labels-dataset/labels'
for file in tqdm(train_files):
    shutil.copy(file, '/kaggle/working/vinbigdata/images/train')
    filename = file.split('/')[-1].split('.')[0]
    
    # nms stuff
    print(filename)
    Create_softnms_box_txt('/kaggle/working/vinbigdata/labels/train',filename)
    
#     shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/vinbigdata/labels/train')
    
for file in tqdm(val_files):
    shutil.copy(file, '/kaggle/working/vinbigdata/images/val')
    filename = file.split('/')[-1].split('.')[0]
    
    # nms stuff
    Create_softnms_box_txt('/kaggle/working/vinbigdata/labels/val',filename)
#     shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/vinbigdata/labels/val')

In [ ]:
import os
path = os.getcwd()#获取当前路径
# print(path)

all_files = [f for f in os.listdir('/kaggle/working/vinbigdata/labels/train' )]#输出根path下的所有文件名到一个列表中
# all_files = [f for f in os.listdir(path )]#输出根path下的所有文件名到一个列表中
#对各个文件进行处理
print(all_files)

# Get Class Name

In [ ]:
class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

# [YOLOv5](https://github.com/ultralytics/yolov5)
![](https://user-images.githubusercontent.com/26833433/98699617-a1595a00-2377-11eb-8145-fc674eb9b1a7.jpg)
![](https://user-images.githubusercontent.com/26833433/90187293-6773ba00-dd6e-11ea-8f90-cd94afc0427f.png)

# YOLOv5 Stuff

In [ ]:
from os import listdir
from os.path import isfile, join
import yaml

cwd = '/kaggle/working/'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('/kaggle/working/vinbigdata/images/train/*'):
        f.write(path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('/kaggle/working/vinbigdata/images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    nc    = 1,
    names = classes
    )

with open(join( cwd , 'vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'vinbigdata.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
# https://www.kaggle.com/ultralytics/yolov5
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd yolov5
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')
%pip install -qr requirements.txt # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/
Image(filename='runs/detect/exp/zidane.jpg', width=600)

## Pretrained Checkpoints:

| Model | AP<sup>val</sup> | AP<sup>test</sup> | AP<sub>50</sub> | Speed<sub>GPU</sub> | FPS<sub>GPU</sub> || params | FLOPS |
|---------- |------ |------ |------ | -------- | ------| ------ |------  |  :------: |
| [YOLOv5s](https://github.com/ultralytics/yolov5/releases/tag/v3.0)    | 37.0     | 37.0     | 56.2     | **2.4ms** | **416** || 7.5M   | 13.2B
| [YOLOv5m](https://github.com/ultralytics/yolov5/releases/tag/v3.0)    | 44.3     | 44.3     | 63.2     | 3.4ms     | 294     || 21.8M  | 39.4B
| [YOLOv5l](https://github.com/ultralytics/yolov5/releases/tag/v3.0)    | 47.7     | 47.7     | 66.5     | 4.4ms     | 227     || 47.8M  | 88.1B
| [YOLOv5x](https://github.com/ultralytics/yolov5/releases/tag/v3.0)    | **49.2** | **49.2** | **67.7** | 6.9ms     | 145     || 89.0M  | 166.4B
| | | | | | || |
| [YOLOv5x](https://github.com/ultralytics/yolov5/releases/tag/v3.0) + TTA|**50.8**| **50.8** | **68.9** | 25.5ms    | 39      || 89.0M  | 354.3B
| | | | | | || |
| [YOLOv3-SPP](https://github.com/ultralytics/yolov5/releases/tag/v3.0) | 45.6     | 45.5     | 65.2     | 4.5ms     | 222     || 63.0M  | 118.0B

# Selecting Models
In this notebok I'm using `v5s`. To select your prefered model just replace `--cfg models/yolov5s.yaml --weights yolov5s.pt` with the following command:
* `v5s` : `--cfg models/yolov5s.yaml --weights yolov5s.pt`
* `v5m` : `--cfg models/yolov5m.yaml --weights yolov5m.pt`
* `v5l` : `--cfg models/yolov5l.yaml --weights yolov5l.pt`
* `v5x` : `--cfg models/yolov5x.yaml --weights yolov5x.pt`

# Train

In [ ]:
hyp_adjust = {'optimizer': 'SGD',  # 优化器['adam', 'SGD', None] if none, default is SGD
       'lr0': 0.01,  # 学习率initial learning rate (SGD=1E-2, Adam=1E-3)
       'momentum': 0.937,  # 学习率动量SGD momentum/Adam beta1
       'weight_decay': 5e-4,  # 权重衰减系数optimizer weight decay
       'giou': 0.05,  # giou损失的系数giou loss gain
       'cls': 0.58,  # 分类损失的系数cls loss gain
       'cls_pw': 1.0,  # 分类BCELoss中正样本的权重cls BCELoss positive_weight
       'obj': 1.0,  # 有无物体损失的系数obj loss gain (*=img_size/320 if img_size != 320)
       'obj_pw': 1.0,  # 有无物体BCELoss中正样本的权重obj BCELoss positive_weight
       'iou_t': 0.20,  # 标签与anchors的iou阈值iou training threshold
       'anchor_t': 4.0,  # 标签的长h宽w/anchor的长h_a宽w_a阈值, 即h/h_a, w/w_a都要在(1/4, 4)之间anchor-multiple threshold
       'fl_gamma': 0.0,  # focal loss gamma, 设为0则表示不使用focal loss(efficientDet default is gamma=1.5)
       # 下面是一些数据增强的系数, 包括颜色空间和图片空间
       'hsv_h': 0.014,  # image HSV-Hue augmentation (fraction)
       'hsv_s': 0.68,  # image HSV-Saturation augmentation (fraction)
       'hsv_v': 0.36,  # image HSV-Value augmentation (fraction)
       'degrees': 0.0,  # image rotation (+/- deg)
       'translate': 0.0,  # image translation (+/- fraction)
       'scale': 0.5,  # image scale (+/- gain)
       'shear': 0.0}

In [ ]:
# # !WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --nosave --cache 
!WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 30 --data /kaggle/working/vinbigdata.yaml --weights yolov5s.pt --cache 

# Class Distribution

In [ ]:

!pip install matplotlib==3.1.3

In [ ]:
plt.figure(figsize = (20,20))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/labels_correlogram.jpg'));

In [ ]:
plt.figure(figsize = (20,20))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/labels.jpg'));

# Batch Image

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (15, 15))
plt.imshow(plt.imread('runs/train/exp/train_batch0.jpg'))

plt.figure(figsize = (15, 15))
plt.imshow(plt.imread('runs/train/exp/train_batch1.jpg'))

plt.figure(figsize = (15, 15))
plt.imshow(plt.imread('runs/train/exp/train_batch2.jpg'))

# GT Vs Pred

In [ ]:
fig, ax = plt.subplots(3, 2, figsize = (2*5,3*5), constrained_layout = True)
for row in range(3):
    ax[row][0].imshow(plt.imread(f'runs/train/exp/test_batch{row}_labels.jpg'))
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f'runs/train/exp/test_batch{row}_labels.jpg', fontsize = 12)
    
    ax[row][1].imshow(plt.imread(f'runs/train/exp/test_batch{row}_pred.jpg'))
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f'runs/train/exp/test_batch{row}_pred.jpg', fontsize = 12)

# (Loss, Map) Vs Epoch

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/results.png'));

# Confusion Matrix

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/confusion_matrix.png'));

# Inference

In [ ]:
!python detect.py --weights 'runs/train/exp/weights/best.pt'\
--img 640\
--conf 0.15\
--iou 0.5\
--source /kaggle/working/vinbigdata/images/val\
--exist-ok

# Inference Plot

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random
import cv2
from glob import glob
from tqdm import tqdm

files = glob('runs/detect/exp/*')
for _ in range(3):
    row = 4
    col = 4
    grid_files = random.sample(files, row*col)
    images     = []
    for image_path in tqdm(grid_files):
        img          = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        images.append(img)

    fig = plt.figure(figsize=(col*5, row*5))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(col, row),  # creates 2x2 grid of axes
                     axes_pad=0.05,  # pad between axes in inch.
                     )

    for ax, im in zip(grid, images):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [ ]:
# shutil.rmtree('/kaggle/working/vinbigdata')
# shutil.rmtree('runs/detect')
# for file in (glob('runs/train/exp/**/*.png', recursive = True)+glob('runs/train/exp/**/*.jpg', recursive = True)):
#     os.remove(file)